# Module 4: Evaluation Metrics for Classification

This notebook contains an exploration into methods for evaluating performance of binary classification problems.

## 4.1 Evaluation metrics: session overview

In the last module we looked at a binary classification problem (churn prediction) where we evaluated the model performance based on accuracy. In this module we will dive into what other metrics can be used to evaluate binary classification model performance.

First we will implement the model we built in the last module, and this will be the starting point of our analysis.


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('../03_Classification/data/Customer-Churn-Dataset.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [4]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

In [5]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [6]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression()
model.fit(X_train, y_train)

/home/shanice/projects/ml-zoomcamp/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [7]:

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)
(y_val == churn_decision).mean()



np.float64(0.801277501774308)

## 4.2 Accuracy and dummy model

In [8]:
len(y_val)

1409

In [9]:
# Accuracy
(y_val == churn_decision).mean()

np.float64(0.801277501774308)

In [10]:
(y_val == churn_decision).sum()

np.int64(1129)

In [11]:
1129/1409

0.801277501774308

What happens to accuracy if we move our decision boundary?

In [13]:
# Create an array of values for thresholds between O and 1
thresholds = np.linspace(0, 1, 21)
thresholds

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [14]:
threshold_accuracies = {}
for t in thresholds:
    churn_decision = (y_pred >= t)
    threshold_accuracies[t] = (y_val == churn_decision).mean()

In [15]:
threshold_accuracies

{np.float64(0.0): np.float64(0.2739531582682754),
 np.float64(0.05): np.float64(0.5088715400993612),
 np.float64(0.1): np.float64(0.5982966643009227),
 np.float64(0.15000000000000002): np.float64(0.6635911994322214),
 np.float64(0.2): np.float64(0.7068843151171044),
 np.float64(0.25): np.float64(0.7374024130589071),
 np.float64(0.30000000000000004): np.float64(0.759403832505323),
 np.float64(0.35000000000000003): np.float64(0.765791341376863),
 np.float64(0.4): np.float64(0.7799858055358411),
 np.float64(0.45): np.float64(0.7934705464868701),
 np.float64(0.5): np.float64(0.801277501774308),
 np.float64(0.55): np.float64(0.7984386089425124),
 np.float64(0.6000000000000001): np.float64(0.7970191625266146),
 np.float64(0.65): np.float64(0.7842441447835344),
 np.float64(0.7000000000000001): np.float64(0.7650816181689141),
 np.float64(0.75): np.float64(0.7437899219304471),
 np.float64(0.8): np.float64(0.7295954577714692),
 np.float64(0.8500000000000001): np.float64(0.7260468417317246),
 np.

In [24]:
threshold_df = pd.DataFrame.from_dict(threshold_accuracies, orient='index',
                                      columns=['accuracy'])

In [25]:
threshold_df

accuracy
0.00  0.273953
0.05  0.508872
0.10  0.598297
0.15  0.663591
0.20  0.706884
0.25  0.737402
0.30  0.759404
0.35  0.765791
0.40  0.779986
0.45  0.793471
0.50  0.801278
0.55  0.798439
0.60  0.797019
0.65  0.784244
0.70  0.765082
0.75  0.743790
0.80  0.729595
0.85  0.726047
0.90  0.726047
0.95  0.726047
1.00  0.726047

In [26]:
sns.lineplot(threshold_df)

<Axes: >

It turns out we used the best threshold i.e. the one that results in the best accuracy.

## 4.3 Confusion Table

## 4.4 Precision and Recall


## 4.5 ROC Curves



## 4.6 ROC AUC

## 4.7 Cross Validation 